<a href="https://colab.research.google.com/github/stasiek-j/SAD2/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import distributions
import sklearn, torch
import numpy as np
torch.manual_seed(42)

In [2]:
K = 3 # Ilość klastrów
N = 2 # Ilu wymiarowe dane 
size = (K, N)

In [3]:
component = distributions.Independent(distributions.Normal(
    torch.randn(size), torch.rand(size)), 1)
mixture = distributions.Categorical(torch.ones(K,))

gmm = distributions.MixtureSameFamily(mixture, component)

In [4]:
gmm.sample()

tensor([0.8017, 0.5883])

In [5]:
d = gmm.sample((10,))
print(d)
d.shape

tensor([[-0.0876,  0.8052],
        [-2.1717,  0.6077],
        [-0.5629,  0.4603],
        [ 0.3422,  0.3295],
        [ 0.8366, -0.2355],
        [-0.1124, -0.1567],
        [ 0.9697, -0.2746],
        [ 1.4971,  0.4033],
        [ 0.9680,  0.1806],
        [ 0.4519,  0.5627]])


torch.Size([10, 2])

In [6]:
gmm.component_distribution

Independent(Normal(loc: torch.Size([3, 2]), scale: torch.Size([3, 2])), 1)

In [7]:
parameters = [(torch.Tensor([.5, .5]), torch.Tensor([[.5, 0], [0, .5]])),
              (torch.Tensor([.4, .4]), torch.Tensor([[.2, 0], [0, .2]]))]

def initialize_params(size):
  f = lambda n: (torch.randn(n), torch.diag(torch.rand(n)))
  return [f(size[1]) for _ in range(size[0])]

parameters = initialize_params(size)
parameters

[(tensor([-0.6556,  0.8576]), tensor([[0.0766, 0.0000],
          [0.0000, 0.8460]])),
 (tensor([ 1.2176, -0.8914]), tensor([[0.6431, 0.0000],
          [0.0000, 0.3908]])),
 (tensor([-2.4752, -0.9316]), tensor([[0.4137, 0.0000],
          [0.0000, 0.6044]]))]

In [8]:
def prob(x, params):
  dis = distributions.MultivariateNormal(*params)
  return dis.log_prob(x)

In [9]:
def e_step(data, params):
  ret = []
  for i in range(data.shape[0]):
    x = data[i, :]
    pclusters = np.array([prob(x, param) for param in params])
    ret.append(np.argmax(pclusters))
  return np.array(ret)


def m_step(data, params, clusters):
  cluster = {i: [] for i in range(len(params))}
  for i in range(len(data)):
    cluster[clusters[i]].append(data[i, :])
  percents = [len(cluster[i])/len(data) for i in range(len(params))]
  print(cluster)
  print(percents)
  pass

In [10]:
post_e = e_step(d, parameters)
post_e

array([0, 2, 0, 1, 1, 0, 1, 1, 1, 1])

In [11]:
m_step(d, parameters, post_e)

{0: [tensor([-0.0876,  0.8052]), tensor([-0.5629,  0.4603]), tensor([-0.1124, -0.1567])], 1: [tensor([0.3422, 0.3295]), tensor([ 0.8366, -0.2355]), tensor([ 0.9697, -0.2746]), tensor([1.4971, 0.4033]), tensor([0.9680, 0.1806]), tensor([0.4519, 0.5627])], 2: [tensor([-2.1717,  0.6077])]}
[0.3, 0.6, 0.1]


In [12]:
d

tensor([[-0.0876,  0.8052],
        [-2.1717,  0.6077],
        [-0.5629,  0.4603],
        [ 0.3422,  0.3295],
        [ 0.8366, -0.2355],
        [-0.1124, -0.1567],
        [ 0.9697, -0.2746],
        [ 1.4971,  0.4033],
        [ 0.9680,  0.1806],
        [ 0.4519,  0.5627]])